In [27]:
import os
import cv2 as cv
import matplotlib.pyplot as plt

In [28]:
def preprocessing(img):
    # convert to grayscale
    preprocessed_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # apply gaussian blur
    preprocessed_img = cv.GaussianBlur(preprocessed_img, (5,5), 0)
    # apply Canny Edge
    preprocessed_img = cv.Canny(image=preprocessed_img, threshold1=25, threshold2=150)
    return preprocessed_img

In [29]:
# Load Folders in data_test
#data_test = for the validation and tracking
data_test_dir = [dir for dir in os.listdir("./data_test/")]

In [30]:
helmetLabels = [
    "DHelmetP1Helmet",
    "DHelmet",
    "DHelmetP1HelmetP2Helmet",
    "DHelmetP0Helmet",
    "DHelmetP0HelmetP1HelmetP2Helmet",
    "DHelmetP0HelmetP1Helmet"
]

In [31]:
os.makedirs("./Helmet_test/", exist_ok=True)
os.makedirs("./NoHelmet_test/", exist_ok=True)

In [32]:
data_counter = 0
for video_dir in range(len(data_test_dir)):
    for frame in range(100):
    # Loading sample image
        img_dir = [i for i in sorted(os.listdir(f"./data_test/{data_test_dir[video_dir]}"))]
        img_path = f"./data_test/{data_test_dir[video_dir]}/{img_dir[frame]}"
        img = cv.imread(img_path)

        # Create a bounding boxes based on annotations file
        boxes_lists = []
        label_texts = []
        annotation_file = f"./annotations/{data_test_dir[video_dir]}.csv"

        with open (annotation_file, "r") as x:
            for lines in x.readlines():
                temp = lines.split(",")
                try:
                    if int(temp[1]) == frame:
                        box = (int(temp[2]), int(temp[3]), int(temp[4]), int(temp[5]))
                        boxes_lists.append(box)

                        label = str(temp[-1])[:-1]
                        label_texts.append(label)
                except:
                    pass


        for i in range(len(boxes_lists)):
            # Crop the images based on bounding boxes (x, y, w, h)
            temp_img = img.copy()
            crop_img = temp_img[boxes_lists[i][1]:boxes_lists[i][1]+boxes_lists[i][3], boxes_lists[i][0]:boxes_lists[i][0]+boxes_lists[i][2]]
            processed_img = preprocessing(crop_img)

            if label_texts[i] in helmetLabels:
                #for validation, preprocessed na ang frames so that faster pag predict
                cv.imwrite(f"./Helmet_test/img{data_counter}.jpg", processed_img)
            cv.imwrite(f"./NoHelmet_test/img{data_counter}.jpg", processed_img)
            data_counter += 1

In [33]:
from keras.models import load_model
import numpy as np

In [34]:
np.set_printoptions(suppress=True)

# Load the model and labels
model = load_model("keras_Model.h5", compile=False)
class_names = open("labels.txt", "r").readlines()

In [35]:
# Checking Accuracy for Helmet Images

helmet_test_files = [img for img in os.listdir("./Helmet_test/")]
helmet_score = 0

for item in helmet_test_files:
    # open the image
    img = cv.imread(f"./Helmet_test/{item}")
    img_crop = cv.resize(img, (224, 224), interpolation=cv.INTER_AREA)
    # Run the model and get the prediction
    img_crop = np.asarray(img_crop, dtype=np.float32).reshape(1, 224, 224, 3)
    img_toPred = (img_crop / 127.5) - 1
    prediction = model.predict(img_toPred, verbose=0)
    index = np.argmax(prediction)
    class_name = int(class_names[index][0])
    if class_name == 0:
        helmet_score += 1

In [36]:
print(f"Helmet Score: {helmet_score}\nTotal Images: {len(helmet_test_files)}\n")
print(f"Accuracy: {(helmet_score / len(helmet_test_files)) * 100}%")

Helmet Score: 91
Total Images: 723

Accuracy: 12.586445366528354%


In [37]:
# Checking Accuracy for NoHelmet Images

NoHelmet_test_files = [img for img in os.listdir("./NoHelmet_test/")]
NoHelmet_score = 0
for item in NoHelmet_test_files:
    # open the image
    img = cv.imread(f"./NoHelmet_test/{item}")
    img_crop = cv.resize(img, (224, 224), interpolation=cv.INTER_AREA)
    # Run the model and get the prediction
    img_crop = np.asarray(img_crop, dtype=np.float32).reshape(1, 224, 224, 3)
    img_toPred = (img_crop / 127.5) - 1
    prediction = model.predict(img_toPred, verbose=0)
    index = np.argmax(prediction)
    class_name = int(class_names[index][0])
    if class_name == 1:
        NoHelmet_score += 1

In [38]:
print(f"NoHelmet Score: {NoHelmet_score}\nTotal Images: {len(NoHelmet_test_files)}\n")
print(f"Accuracy: {(NoHelmet_score / len(NoHelmet_test_files)) * 100}%")

NoHelmet Score: 1684
Total Images: 1800

Accuracy: 93.55555555555556%


In [39]:
# Calculating for Total Accuracy
total_score = helmet_score + NoHelmet_score
total_test_files = len(helmet_test_files) + len(NoHelmet_test_files)

accuracy = (total_score/total_test_files) * 100
print(f"Accuracy: {accuracy}%")

Accuracy: 70.35275465715418%
